# Neural Network Fitting and Benchmarking

## Imports

In [32]:
import os
from m3util.viz.printing import printer
from m3util.ml.optimizers.TrustRegion import TRCG
from m3util.viz.style import set_style
from m3util.ml.rand import set_seeds
from belearn.dataset.dataset import BE_Dataset
from belearn.functions.sho import SHO_nn
from belearn.nn.nn import batch_training
from datetime import datetime

from autophyslearn.postprocessing.complex import ComplexPostProcessor
from autophyslearn.spectroscopic.nn import Multiscale1DFitter, Model


In [38]:
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

# Specify the filename and the path to save the file
filename = "data_raw.h5"
save_path = "./Data"


optimizer_TR = {"name": "TRCG", "optimizer": TRCG, "radius": 5, "device": "cuda", "ADAM_epochs": 2}
# optimizers = [ 'Adam', optimizer_TR]
optimizers = [optimizer_TR]
noise_list = [0, 1, 2, 3, 4, 5, 6, 7, 8]
batch_size = [500, 1000, 5000, 10000]
epochs = [5]
seed = [41, 43, 44, 45, 46]
early_stopping_time = 60*3
basepath_postfix = 'nn_benchmarks_noise'

# Original filename
csv_name = 'nn_benchmarks_noise.csv'

printing = printer(basepath='./Figures/')

set_style("printing")
set_seeds(seed=42)

data_path = save_path + "/" + filename

printing set for seaborn
Pytorch seed was set to 42
Numpy seed was set to 42
tensorflow seed was set to 42


In [39]:
# instantiate the dataset object
dataset = BE_Dataset(data_path, SHO_fit_func_LSQF=SHO_nn)

# print the contents of the file
dataset.print_be_tree()

# Get the current date and time
current_datetime = datetime.now()

# Format the date and time in a 'pretty' format (e.g., YYYY-MM-DD_HH-MM-SS)
formatted_datetime = current_datetime.strftime('%Y-%m-%d_%H-%M-%S')

/
├ Measurement_000
  ---------------
  ├ Channel_000
    -----------
    ├ Bin_FFT
    ├ Bin_Frequencies
    ├ Bin_Indices
    ├ Bin_Step
    ├ Bin_Wfm_Type
    ├ Excitation_Waveform
    ├ Noise_Floor
    ├ Noisy_Data_1
    ├ Noisy_Data_2
    ├ Noisy_Data_3
    ├ Noisy_Data_4
    ├ Noisy_Data_5
    ├ Noisy_Data_6
    ├ Noisy_Data_7
    ├ Noisy_Data_8
    ├ Position_Indices
    ├ Position_Values
    ├ Raw_Data
    ├ Spatially_Averaged_Plot_Group_000
      ---------------------------------
      ├ Bin_Frequencies
      ├ Max_Response
      ├ Mean_Spectrogram
      ├ Min_Response
      ├ Spectroscopic_Parameter
      ├ Step_Averaged_Response
    ├ Spatially_Averaged_Plot_Group_001
      ---------------------------------
      ├ Bin_Frequencies
      ├ Max_Response
      ├ Mean_Spectrogram
      ├ Min_Response
      ├ Spectroscopic_Parameter
      ├ Step_Averaged_Response
    ├ Spectroscopic_Indices
    ├ Spectroscopic_Values
    ├ UDVS
    ├ UDVS_Indices
├ Noisy_Data_1_SHO_Fit
  --------

In [40]:
basepath = f'{formatted_datetime}_{basepath_postfix}'

# # Usage example
# system_info = SystemInfo()
# cpu_info, gpu_info = system_info.get_system_info()
# system_info.save_to_file("Trained Models/" + basepath,
#                          "system_info.txt", cpu_info, gpu_info)

batch_training(dataset, optimizers, noise_list, batch_size, epochs,
               seed,
               write_CSV="Batch_Trainging_SpeedTest.csv",
               basepath=basepath, 
               early_stopping_time=early_stopping_time)

The type is <class 'dict'>
Pytorch seed was set to 41
Numpy seed was set to 41
tensorflow seed was set to 41
Working on combination: SHO_TRCG_noise_0_batch_size_500_seed_41
Using GPU NVIDIA GeForce RTX 3090
Saving to Trained Models/SHO Fitter//2024-09-12_18-31-42_nn_benchmarks_noise/
Pytorch seed was set to 41
Numpy seed was set to 41
tensorflow seed was set to 41
Adam
epoch : 1/5, recon loss = 0.06054313
--- 17.516742944717407 seconds ---
Epoch 1, Learning Rate: 0.001
Adam
epoch : 2/5, recon loss = 0.03451453
--- 17.219754219055176 seconds ---
Epoch 2, Learning Rate: 0.001
Adam
epoch : 3/5, recon loss = 0.03421530
--- 17.368999242782593 seconds ---
Epoch 3, Learning Rate: 0.001
Trust Region CG
epoch : 4/5, recon loss = 0.03368135
--- 912.2405369281769 seconds ---
Epoch 4, Learning Rate: 0.001
The type is <class 'dict'>
Pytorch seed was set to 43
Numpy seed was set to 43
tensorflow seed was set to 43
Working on combination: SHO_TRCG_noise_0_batch_size_500_seed_43
Using GPU NVIDIA GeFor

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB. GPU 0 has a total capacity of 23.68 GiB of which 1.81 MiB is free. Including non-PyTorch memory, this process has 23.68 GiB memory in use. Of the allocated memory 23.29 GiB is allocated by PyTorch, and 73.73 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
import itertools

# Generate all combinations
combinations = list(itertools.product(
    optimizers, noise_list, batch_size, epochs, seed))

for i, training in enumerate(combinations):

    optimizer_ = training[0]
    noise_ = training[1]
    seed_ = training[4]

    print(i, optimizer_, noise_, seed_)